# I. Validating Class

Random verification of n-dimensional Gaussian models in IMQ kernel and KGM kernel.

In [1]:
from stein_pi_thinning.target import PiTargetAuto, PiTargetIMQ, PiTargetKGM, PiTargetCentKGM

import numpy as np
from scipy.stats import wishart
from jax import numpy as jnp

np.random.seed(1234)

In [2]:
dim = 4

x_test = np.random.normal(size=dim)
mean = np.random.normal(size=dim)
cov = wishart.rvs(dim + 10, np.eye(dim), size=1)

print("x_test:", x_test)
print("mean  :", mean)
print("cov   :", cov)

x_test: [ 0.47143516 -1.19097569  1.43270697 -0.3126519 ]
mean  : [-0.72058873  0.88716294  0.85958841 -0.6365235 ]
cov   : [[11.68150068  0.05364739 -7.66509583  3.39029399]
 [ 0.05364739 12.34410368  4.00531349  3.36496236]
 [-7.66509583  4.00531349 25.21091916 -9.90588931]
 [ 3.39029399  3.36496236 -9.90588931 10.75391052]]


In [3]:
# Test manual IMQ class
log_p = lambda x:-np.log(2*np.pi) - 0.5*np.log(np.linalg.det(cov)) - 0.5*(x-mean)@np.linalg.inv(cov)@(x-mean)
grad_log_p = lambda x: -np.linalg.inv(cov)@(x-mean)
hess_log_p = lambda x: -np.linalg.inv(cov)
linv = np.linalg.inv(cov)

test_manual_imq = PiTargetIMQ(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv)

print("log-pdf   :", test_manual_imq.log_p(x_test))
print("glog-pdf  :", test_manual_imq.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_imq.hess_log_p(x_test))
print("log-q-pdf :", test_manual_imq.log_q(x_test))
print("glog-q-pdf:", test_manual_imq.grad_log_q(x_test))

log-pdf   : -7.153352187026759
glog-pdf  : [-0.17921809  0.33648779 -0.25339807 -0.31232095]
hlog-pdf  : [[-0.10860438  0.013307   -0.03654333 -0.0035867 ]
 [ 0.013307   -0.12428333  0.05865073  0.08871951]
 [-0.03654333  0.05865073 -0.09838846 -0.09746129]
 [-0.0035867   0.08871951 -0.09746129 -0.20939533]]
log-q-pdf : -7.235925371128592
glog-q-pdf: [-0.13873336  0.23413024 -0.15708047 -0.17007607]


In [4]:
# Test auto IMQ class
log_p = lambda x:-jnp.log(2*jnp.pi) - 0.5*jnp.log(jnp.linalg.det(cov)) - 0.5*(x-mean)@jnp.linalg.inv(cov)@(x-mean)
linv = jnp.linalg.inv(cov)
imq_kernel = lambda x, y: (1 + (x - y)@linv@(x - y))**(-0.5)

test_auto_imq = PiTargetAuto(log_p=log_p, base_kernel=imq_kernel)

print("log-pdf   :", test_auto_imq.log_p(x_test))
print("glog-pdf  :", test_auto_imq.grad_log_p(x_test))
print("hlog-pdf  :", test_auto_imq.hess_log_p(x_test))
print("log-q-pdf :", test_auto_imq.log_q(x_test))
print("glog-q-pdf:", test_auto_imq.grad_log_q(x_test))

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


log-pdf   : -7.1533523
glog-pdf  : [-0.17921811  0.33648777 -0.2533981  -0.31232098]
hlog-pdf  : [[-0.10860439  0.013307   -0.03654334 -0.0035867 ]
 [ 0.013307   -0.12428331  0.05865073  0.08871952]
 [-0.03654334  0.05865073 -0.09838847 -0.09746131]
 [-0.0035867   0.08871952 -0.09746131 -0.20939535]]
log-q-pdf : -7.235925
glog-q-pdf: [-0.13873339  0.23413023 -0.15708049 -0.17007609]


In [5]:
# Test manual KGM class
log_p = lambda x:-np.log(2*np.pi) - 0.5*np.log(np.linalg.det(cov)) - 0.5*(x-mean)@np.linalg.inv(cov)@(x-mean)
grad_log_p = lambda x: -np.linalg.inv(cov)@(x-mean)
hess_log_p = lambda x: -np.linalg.inv(cov)
linv = np.linalg.inv(cov)

test_manual_kgm = PiTargetKGM(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv, s=3.0)

print("log-pdf   :", test_manual_kgm.log_p(x_test))
print("glog-pdf  :", test_manual_kgm.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_kgm.hess_log_p(x_test))
print("log-q-pdf :", test_manual_kgm.log_q(x_test))
print("glog-q-pdf:", test_manual_kgm.grad_log_q(x_test))

log-pdf   : -7.153352187026759
glog-pdf  : [-0.17921809  0.33648779 -0.25339807 -0.31232095]
hlog-pdf  : [[-0.10860438  0.013307   -0.03654333 -0.0035867 ]
 [ 0.013307   -0.12428333  0.05865073  0.08871951]
 [-0.03654333  0.05865073 -0.09838846 -0.09746129]
 [-0.0035867   0.08871951 -0.09746129 -0.20939533]]
log-q-pdf : -6.832792089900934
glog-q-pdf: [-0.06523543  0.12608668 -0.05548275 -0.1164513 ]


In [6]:
# Test auto KGM class
s = 3.0
linv = jnp.linalg.inv(cov)
log_p = lambda x:-jnp.log(2*jnp.pi) - 0.5*jnp.log(jnp.linalg.det(cov)) - 0.5*(x-mean)@jnp.linalg.inv(cov)@(x-mean)

kgm_kernel = lambda x, y: (1 + x@linv@x)**((s-1)/2) *\
        (1 + y@linv@y)**((s-1)/2) *\
        (1 + (x-y)@linv@(x-y))**(-0.5) +\
        (1 + x@linv@y )/( jnp.sqrt(1+x@linv@x) * jnp.sqrt(1+y@linv@y) )

test_auto_kgm = PiTargetAuto(log_p=log_p, base_kernel=kgm_kernel)

print("log-pdf   :", test_auto_kgm.log_p(x_test))
print("glog-pdf  :", test_auto_kgm.grad_log_p(x_test))
print("hlog-pdf  :", test_auto_kgm.hess_log_p(x_test))
print("log-q-pdf :", test_auto_kgm.log_q(x_test))
print("glog-q-pdf:", test_auto_kgm.grad_log_q(x_test))

log-pdf   : -7.1533523
glog-pdf  : [-0.17921811  0.33648777 -0.2533981  -0.31232098]
hlog-pdf  : [[-0.10860439  0.013307   -0.03654334 -0.0035867 ]
 [ 0.013307   -0.12428331  0.05865073  0.08871952]
 [-0.03654334  0.05865073 -0.09838847 -0.09746131]
 [-0.0035867   0.08871952 -0.09746131 -0.20939535]]
log-q-pdf : -6.8327923
glog-q-pdf: [-0.06523542  0.12608667 -0.05548273 -0.11645128]


In [7]:
# Test manual Centralized KGM class
log_p = lambda x:-np.log(2*np.pi) - 0.5*np.log(np.linalg.det(cov)) - 0.5*(x-mean)@np.linalg.inv(cov)@(x-mean)
grad_log_p = lambda x: -np.linalg.inv(cov)@(x-mean)
hess_log_p = lambda x: -np.linalg.inv(cov)
linv = np.linalg.inv(cov)

test_manual_centkgm = PiTargetCentKGM(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv, s=3.0, x_map=mean)

print("log-pdf   :", test_manual_centkgm.log_p(x_test))
print("glog-pdf  :", test_manual_centkgm.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_centkgm.hess_log_p(x_test))
print("log-q-pdf :", test_manual_centkgm.log_q(x_test))
print("glog-q-pdf:", test_manual_centkgm.grad_log_q(x_test))

log-pdf   : -7.153352187026759
glog-pdf  : [-0.17921809  0.33648779 -0.25339807 -0.31232095]
hlog-pdf  : [[-0.10860438  0.013307   -0.03654333 -0.0035867 ]
 [ 0.013307   -0.12428333  0.05865073  0.08871951]
 [-0.03654333  0.05865073 -0.09838846 -0.09746129]
 [-0.0035867   0.08871951 -0.09746129 -0.20939533]]
log-q-pdf : -6.600675390113612
glog-q-pdf: [-0.02890968  0.04828224 -0.0319816  -0.03406256]


In [8]:
# Test auto Centralized KGM class
s = 3.0
x_map = mean
linv = jnp.linalg.inv(cov)
log_p = lambda x:-jnp.log(2*jnp.pi) - 0.5*jnp.log(jnp.linalg.det(cov)) - 0.5*(x-mean)@jnp.linalg.inv(cov)@(x-mean)

centkgm_kernel = lambda x, y: (1 + (x-x_map)@linv@(x-x_map))**((s-1)/2) * (1 + (y-x_map)@linv@(y-x_map))**((s-1)/2)\
      * ((1 + (x-y)@linv@(x-y).T )**(-0.5) +\
            (1 + (x-x_map)@linv@(y-x_map).T)/( (1+(x-x_map)@linv@(x-x_map).T)**(s/2) * (1+(y-x_map)@linv@(y-x_map).T)**(s/2) ))

test_auto_centkgm = PiTargetAuto(log_p=log_p, base_kernel=centkgm_kernel)

print("log-pdf   :", test_auto_centkgm.log_p(x_test))
print("glog-pdf  :", test_auto_centkgm.grad_log_p(x_test))
print("hlog-pdf  :", test_auto_centkgm.hess_log_p(x_test))
print("log-q-pdf :", test_auto_centkgm.log_q(x_test))
print("glog-q-pdf:", test_auto_centkgm.grad_log_q(x_test))

log-pdf   : -7.1533523
glog-pdf  : [-0.17921811  0.33648777 -0.2533981  -0.31232098]
hlog-pdf  : [[-0.10860439  0.013307   -0.03654334 -0.0035867 ]
 [ 0.013307   -0.12428331  0.05865073  0.08871952]
 [-0.03654334  0.05865073 -0.09838847 -0.09746131]
 [-0.0035867   0.08871952 -0.09746131 -0.20939535]]
log-q-pdf : -6.6006756
glog-q-pdf: [-0.02890968  0.04828227 -0.03198163 -0.03406259]


# II. Verifying Non-Python Objects

Verifying the Availability of Black-Box Log-Posterior Distributions and their Gradients Compiled by Stan

In [9]:
import os
import json
import pandas as pd
from stein_pi_thinning.util import flat
import bridgestan as bs
from posteriordb import PosteriorDatabase

In [10]:
# Load DataBase Locally
pdb_path = os.path.join(os.getcwd(), "posteriordb/posterior_database")
my_pdb = PosteriorDatabase(pdb_path)

# Load Dataset
posterior = my_pdb.posterior("eight_schools-eight_schools_centered")
stan = posterior.model.stan_code_file_path()
data = json.dumps(posterior.data.values())
model = bs.StanModel.from_stan_file(stan, data)

# Gold Standard
gs = posterior.reference_draws()
df = pd.DataFrame(gs)
gs_chains = np.zeros((sum(flat(posterior.information['dimensions'].values())),\
                       posterior.reference_draws_info()['diagnostics']['ndraws']))
for i in range(len(df.keys())):
    s = []
    for j in range(len(df[df.keys()[i]])):
        s += df[df.keys()[i]][j]
    gs_chains[i, :] = s
linv = np.linalg.inv(np.cov(gs_chains))

# Extract log-P-pdf and its gradient
log_p = model.log_density
grad_log_p = lambda x: model.log_density_gradient(x)[1]
hess_log_p = lambda x: model.log_density_hessian(x)[2]

In [11]:
# Test manual IMQ class
dim = model.param_num()
x_test = np.random.normal(size=dim)

test_manual_imq = PiTargetIMQ(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv)

print("log-pdf   :", test_manual_imq.log_p(x_test))
print("glog-pdf  :", test_manual_imq.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_imq.hess_log_p(x_test))
print("log-q-pdf :", test_manual_imq.log_q(x_test))
print("glog-q-pdf:", test_manual_imq.grad_log_q(x_test))

log-pdf   : -9.01998466982852
glog-pdf  : [-0.17749739 -0.40994467 -0.89068497  0.09812123 -0.56727961  0.7459595
  0.07248543 -0.70806775  2.31392932 -2.28824916]
hlog-pdf  : [[-0.51366436  0.          0.          0.          0.          0.
   0.          0.          0.50921991  0.60216437]
 [ 0.         -0.51921991  0.          0.          0.          0.
   0.          0.          0.50921991  0.96882056]
 [ 0.          0.         -0.51312616  0.          0.          0.
   0.          0.          0.50921991  1.74763437]
 [ 0.          0.          0.         -0.51748438  0.          0.
   0.          0.          0.50921991 -0.07278286]
 [ 0.          0.          0.          0.         -0.52156559  0.
   0.          0.          0.50921991  1.09318752]
 [ 0.          0.          0.          0.          0.         -0.51748438
   0.          0.          0.50921991 -1.44535656]
 [ 0.          0.          0.          0.          0.          0.
  -0.51921991  0.          0.50921991  0.2186913

In [12]:
# Test manual KGM class
dim = model.param_num()
x_test = np.random.normal(size=dim)

test_manual_kgm = PiTargetKGM(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv, s=3.0)

print("log-pdf   :", test_manual_kgm.log_p(x_test))
print("glog-pdf  :", test_manual_kgm.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_kgm.hess_log_p(x_test))
print("log-q-pdf :", test_manual_kgm.log_q(x_test))
print("glog-q-pdf:", test_manual_kgm.grad_log_q(x_test))

log-pdf   : -13.198300114983885
glog-pdf  : [-2.8959757  -2.94113106 -2.46003717  0.6744793  -0.16450099  1.24965883
 -2.2049274  -7.1567994  16.32757001 19.82672475]
hlog-pdf  : [[ -3.10906611   0.           0.           0.           0.
    0.           0.           0.           3.10462166   6.03152848]
 [  0.          -3.11462166   0.           0.           0.
    0.           0.           0.           3.10462166   6.02134335]
 [  0.           0.          -3.10852791   0.           0.
    0.           0.           0.           3.10462166   4.88988905]
 [  0.           0.           0.          -3.11288613   0.
    0.           0.           0.           3.10462166  -1.23123808]
 [  0.           0.           0.           0.          -3.11696734
    0.           0.           0.           3.10462166   0.30123129]
 [  0.           0.           0.           0.           0.
   -3.11288613   0.           0.           3.10462166  -2.47745328]
 [  0.           0.           0.           0.      

In [13]:
# Test manual Centralized KGM class
dim = model.param_num()
x_test = np.random.normal(size=dim)
x_map = model.param_unconstrain(np.mean(gs_chains, axis=1))

test_manual_centkgm = PiTargetCentKGM(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv, s=3.0, x_map=x_map)

print("log-pdf   :", test_manual_centkgm.log_p(x_test))
print("glog-pdf  :", test_manual_centkgm.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_centkgm.hess_log_p(x_test))
print("log-q-pdf :", test_manual_centkgm.log_q(x_test))
print("glog-q-pdf:", test_manual_centkgm.grad_log_q(x_test))

log-pdf   : -125.29449874971975
glog-pdf  : [  14.62998483   52.65407133   10.6785506    31.39132912   17.61179453
   14.83556516    1.7150771    11.31376635 -154.37989176  255.32649628]
hlog-pdf  : [[ -18.02692961    0.            0.            0.            0.
     0.            0.            0.           18.02248516  -29.01140202]
 [   0.          -18.03248516    0.            0.            0.
     0.            0.            0.           18.02248516 -105.10664311]
 [   0.            0.          -18.02639141    0.            0.
     0.            0.            0.           18.02248516  -21.38247457]
 [   0.            0.            0.          -18.03074963    0.
     0.            0.            0.           18.02248516  -62.65212674]
 [   0.            0.            0.            0.          -18.03483084
     0.            0.            0.           18.02248516  -35.24490276]
 [   0.            0.            0.            0.            0.
   -18.03074963    0.            0.         